In [1]:
import arcpy
import os
from pathlib import Path
import pickle
import sys

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import numpy as np
import pandas as pd

sys.path.append('../../src')
from geoai_retail.analysis import get_add_new_closest_dataframe

%load_ext autoreload
%autoreload 2

In [2]:
data = Path(os.path.abspath('../../data'))
interim_dir = data/'interim'
raw_dir = data/'raw'
raw_gdb = raw_dir/'raw.gdb'

origin_fc = str(raw_gdb/'sea_block_group')
origin_id_fld = 'ID'

dest_fc = str(raw_gdb/'sea_ace')
dest_id_fld = 'LOCNUM'

comp_fc = raw_gdb/'sea_ace_comp'
comp_id_fld = 'LOCNUM'

closest_brand = interim_dir/'closest_store.csv'
closest_comp = interim_dir/'closest_competition.csv'
origin_demographics = interim_dir/'origin_enrich_all.csv'
inrix_trips = interim_dir/'cust_count_inrix.csv'
training_data = interim_dir/'training_data.csv'

scratch_gdb = Path(arcpy.env.scratchGDB)
arcpy.env.overwriteOutput = True

In [3]:
out_sr = arcpy.SpatialReference(3857)
in_desc = arcpy.Describe(origin_fc)

transformation_lst = arcpy.ListTransformations(in_desc.spatialReference, out_sr)

if len(transformation_lst):
    prj_fc = arcpy.management.Project(origin_fc, str(scratch_gdb/'temp_fc'), out_sr, transformation_lst[0])
else:
    prj_fc = arcpy.management.Project(origin_fc, str(scratch_gdb/'temp_fc'), out_sr)

In [4]:
with arcpy.da.SearchCursor(prj_fc, 'SHAPE@') as search_cur:
    for row in search_cur:
        geom = row[0]
        break

In [6]:
from scipy.stats import iqr

In [15]:
area_lst = [row[0].area for row in arcpy.da.SearchCursor(prj_fc, 'SHAPE@')]
area_lst.sort()

In [18]:
f'{np.quantile(area_lst, 0.25):,}'

'994,554.7260704016'

In [19]:
f'{np.quantile(area_lst, 0.75):,}'

'3,692,248.390682234'

In [5]:
area_mean = np.mean([row[0].area for row in arcpy.da.SearchCursor(prj_fc, 'SHAPE@')])
area_mean

13624395.891716829

In [48]:
hex_fc = arcpy.management.GenerateTessellation(str(interim_dir/'interim.gdb/sea_hexbin_02'), arcpy.Describe(prj_fc).extent, 'HEXAGON', f'{area_mean}')

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'